In [ ]:
import pandas as pd

In [ ]:
hyperparam = {
    "valid_pct": 0.3, # 30% of the original dataset is designated as validation set
    "num_sentences_compare": 500 # each sentence in the training and the validation set will be matched with 500 other random sentences from the same set.
}

In [ ]:
base_path = "../input/commonlitreadabilityprize/"
train_df = pd.read_csv(base_path+"train.csv")
test_df = pd.read_csv(base_path+"test.csv")
sample_submission = pd.read_csv(base_path+"sample_submission.csv")

In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)[['excerpt', 'target']]

valid_qty = int(len(train_df) * hyperparam["valid_pct"])
valid_df = train_df[:valid_qty]
train_df = train_df[valid_qty:]
train_df = train_df.sort_values(by=['target']).reset_index(drop=True)

In [ ]:
def sentence_comparison(df, num_comparisons=300, sort=True):
    len_df = len(df)
    comparison_df = pd.concat(
        [
            df.loc[df.index.repeat(num_comparisons)].rename(columns={"excerpt": "excerpt1", "target": "target1"}).reset_index(drop=True),
            df.sample(n=len_df*num_comparisons, replace=True).rename(columns={"excerpt": "excerpt2", "target": "target2"}).reset_index(drop=True)
        ],
        axis=1
    )
    comparison_df['delta_target'] = comparison_df['target1'] - comparison_df['target2']
    comparison_df['target1_geq_target2'] = comparison_df['target1'] >= comparison_df['target2']
    if sort:
        comparison_df = comparison_df.sample(frac=1).reset_index(drop=True)
    return comparison_df

In [ ]:
train_ds = sentence_comparison(train_df, hyperparam["num_sentences_compare"])
valid_ds = sentence_comparison(valid_df, hyperparam["num_sentences_compare"])

In [ ]:
train_ds.to_csv('complexity_comparison_train.csv')

In [ ]:
valid_ds.to_csv('complexity_comparison_valid.csv')